<a href="https://colab.research.google.com/github/ViniciusPereiradeSouzaSuguimoto/atividades/blob/main/Treinamento_de_raspagem_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalando bibliotecas
!pip install scrapy
!pip install crochet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 6.9 MB/s eta 0:00:00


In [2]:
# text cleaning
import re
import scrapy
import logging
from scrapy.crawler import CrawlerRunner
# Reactor restart
from crochet import setup, wait_for
setup()

In [7]:
class mercadolivreSpider(scrapy.Spider):
    name = 'mercadolivre_spider'
    allowed_domains = ['mercadolivre.com.br']
    start_urls = ['https://www.mercadolivre.com.br/ofertas',]

    custom_settings = {
        'LOG_LEVEL': 'DEBUG',
        'FEED_FORMAT':'csv',                            # Used for pipeline
        'FEED_URI': 'mercadolivre.csv',                       # Used for pipeline
        'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
    }

    def parse(self, response):
        for link in response.css(".promotion-item__link-container::attr(href)").getall():
            #text_page = f"https://www.mercadolivre.com.br/ofertas{link}"
            print(link)
            yield scrapy.Request(link, callback=self.parse_text)

    def parse_text(self, response):
        content = ""
        #Atualizando para novo leiaute do site o tempo
        for line in response.css('.ui-pdp-description__content::text').getall() :
          content = content + "".join(line.strip()) + "\n"
          yield {"conteudo": content}


In [8]:
@wait_for(10)
def run_spider():
  setup()
  crawler = CrawlerRunner()
  crawler.crawl(mercadolivreSpider)

In [9]:
run_spider()


It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.


In [10]:
import pandas as pd

df = pd.read_csv("mercadolivre.csv")
df

,conteudo
0,"A PERFORMANCE QUE VOCÊ QUER, O DESIGN QUE VOCÊ..."
1,"A PERFORMANCE QUE VOCÊ QUER, O DESIGN QUE VOCÊ..."
2,"A PERFORMANCE QUE VOCÊ QUER, O DESIGN QUE VOCÊ..."
3,"A PERFORMANCE QUE VOCÊ QUER, O DESIGN QUE VOCÊ..."
4,"A PERFORMANCE QUE VOCÊ QUER, O DESIGN QUE VOCÊ..."
...,...
524,A Samsung está fielmente comprometida em forne...
525,A Samsung está fielmente comprometida em forne...
526,A Samsung está fielmente comprometida em forne...
527,A Samsung está fielmente comprometida em forne...


No charts were generated by quickchart


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def histogram(df, colname, num_bins=20, figscale=1):
  from matplotlib import pyplot as plt
  df[colname].plot(kind='hist', bins=num_bins, title=colname, figsize=(8*figscale, 4*figscale))
  plt.gca().spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = histogram(_df_1, *['index'], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_2, *['index', 'count()', None], **{})
chart